# Refinitiv Data Platform Library for Python
## Content - Pricing - Asynchronous calls

This notebook demonstrates how to retrieve Pricing snapshot data from the Refinitiv Data Platform.

#### Learn more

To learn more about the Refinitiv Data Platform Data Libraries just connect to the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/iam/login) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like [Quick Start guides](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/quick-start), [Tutorials](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/learning), [Documentation](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/docs) and much more.  

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.thomsonreuters.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 


## Import the library and load credentials

Credentials used by this notebook are stored in the ./credentials.ipynb. Please edit ./credentials.ipynb to set your credentials and run the next cell to continue with this scripts

In [1]:
import refinitiv.dataplatform as rdp
import asyncio

%run ./credentials.ipynb

## Open the session of your choice

Depending on the access point your application uses to connect to the Refinitiv Data Platform, it needs to call one of the following functions to create and open a session to the platform. This session will become the default session used by all other function calls.

#### Either

Create and open a Desktop session to connect to the Refinitiv Data Platform pvia Eikon 4 or the Refinitiv Workspace.

In [3]:
# The current implementation of the library does not allow Pricing data retrieval via a DesktopSession  

# rdp.open_desktop_session(APP_KEY)

#### or

Create and open a Platform session to connect directly to the Refinitiv Data Platform. 

In [2]:
rdp.open_platform_session(
    APP_KEY, 
    rdp.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    )
)

#### or

Create and open a Deployed Platform session to connect directly to a deployed enterprise platform (a.k.a.TREP). 

In [1]:
# The current implementation of the library does not allow News retrieval via DeployedPlatformSession  

#rdp.open_deployed_platform_session(
#    APP_KEY,
#    DEPLOYED_PLATFORM_HOST,
#    DEPLOYED_PLATFORM_USER_NAME
#)

## Get Price snapshots
### With synchronized parallel calls

In [7]:
tasks = asyncio.gather(
    rdp.Pricing.get_snapshot_async(
        universe = ['GOOG.O', 'AAPL.O', 'FB.O', 'AMZN.O'], 
        fields   = ['BID','ASK']
    ),
    rdp.Pricing.get_snapshot_async(
        universe = ['EUR=', 'GBP=', 'JPY=', 'CAD='], 
        fields   = ['TIMACT','BID','ASK','NETCHNG_1','HIGH_1', 'LOW_1', 'OPEN_PRC', 'HST_CLOSE','DSPLY_NAME']
    )
)


asyncio.get_event_loop().run_until_complete(tasks)
gafa_response, fx_response = tasks._result

display(gafa_response.data.df)
display(fx_response.data.df)

,Instruments,BID,ASK
0,GOOG.O,#N/P,#N/P
1,AAPL.O,#N/P,#N/P
2,FB.O,#N/P,#N/P
3,AMZN.O,#N/P,#N/P


,Instruments,TIMACT,BID,ASK,NETCHNG_1,HIGH_1,LOW_1,OPEN_PRC,HST_CLOSE,DSPLY_NAME
0,EUR=,13:16:00,1.1113,1.1117,0.0002,1.1144,1.1114,1.1111,1.1111,BARCLAYS LON
1,GBP=,13:16:00,1.3066,1.3070,-0.0010,1.3131,1.3056,1.3077,1.3076,BARCLAYS LON
2,JPY=,13:16:00,109.4800,109.5200,-0.0500,109.6700,109.4600,109.5400,109.5300,NEDBANK LTD JHB
3,CAD=,13:16:00,1.3120,1.3124,0.0007,1.3126,1.3108,1.3110,1.3113,BARCLAYS LON


## Close the session

In [8]:
rdp.close_session()